## LLMs and Chat Models

In [14]:
from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI

llm = OpenAI()
chat = ChatOpenAI()

# a = llm.predict("How many planets are there?") // text-davinci-003 2024.01.04 out of service -> 실행 안 됨
b = chat.predict("How many planets are there?") # In service gpt-3.5-turbo

b

'There are eight planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.'

## Predict Messages

In [15]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(
    temperature=0.1 # temperature가 높을수록 창의력이 떨어짐, 답변의 활용도가 낮음
)

WARNING! chat is not default parameter.
                    chat was transferred to model_kwargs.
                    Please confirm that chat is what you intended.


In [22]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [
    SystemMessage(content="You are a geography expert. And you only reply in Italian."),
    AIMessage(content = "Ciao, mi chiam Paolo!"),
    HumanMessage(content="What is the distance between Mexico and Thailand. Also, what is your name?")
]

chat.predict_messages(messages)

TypeError: Object of type ChatOpenAI is not JSON serializable

## FewShoutPromptTemplate

In [30]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate # 질문을 받고 답변을 할 수 있는 PROMPT를 제작하는 모듈
from langchain.prompts.few_shot import FewShotPromptTemplate # AI가 답변하는 형식을 학습시킬 수 있는 모듈 -> 대화 방식을 데이터 베이스에 저장하여 가지고 오면 편함
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ]
)

t = PromptTemplate(
    template="what is the capital of {country}",
    input_variables=['country'],
)

t.format(country="France")

'what is the capital of France'

In [34]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler


chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ]
)

examples = [
{
    "question": "what do you know about Italy?",
    "answer": """
    Here is what I know:
    Capital : Rome
    Language: Italian
    Good: Pizza and Pasta
    Currency: Euro"""
},
{
    "question": "what do you know about Greece?",
    "answer": """
    I know this:
    Capital: Athens
    Language: Greek
    Good:Souvlaki and Feta Cheese
    Currency: Euro"""
},
{
    "question": "what do you kow about France?",
    "answer": """I know this:
    Capital : Paris
    Language: French
    Food: Wine and Cheese
    Currency: Euro"""
}
]
# chat.predict("what do you know about France?")

example_prompt = PromptTemplate.from_template("Human: {question}\nAI{answer}")

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples = examples,
    suffix="Human: What do you know about{country}?",
    input_variables=['country']
)
                                              

prompt.format(country="Germany")

chain = prompt | chat

chain.invoke({
    "country" : "Turkey"
})


AI
    I know this:
    Capital: Ankara
    Language: Turkish
    Food: Kebab and Baklava
    Currency: Turkish Lira

AIMessageChunk(content='AI\n    I know this:\n    Capital: Ankara\n    Language: Turkish\n    Food: Kebab and Baklava\n    Currency: Turkish Lira')